# Metin Ön İşleme (Preprocessing)

Bu notebook, bns-nlp-engine'in preprocessing özelliklerini detaylı olarak gösterir.

In [ ]:
import asyncio
from bnsnlp.preprocess import TurkishPreprocessor
from bnsnlp.preprocess.normalizer import TurkishNormalizer
from bnsnlp.preprocess.tokenizer import Tokenizer
from bnsnlp.preprocess.stopwords import StopWords
from bnsnlp.preprocess.lemmatizer import TurkishLemmatizer

## 1. Temel Preprocessing

In [ ]:
# Basit örnek
preprocessor = TurkishPreprocessor({
    'lowercase': True,
    'remove_punctuation': True,
    'remove_stopwords': False,
    'lemmatize': False
})

text = "Merhaba DÜNYA! Bu bir TEST metnidir."
result = await preprocessor.process(text)

print(f"Orijinal: {text}")
print(f"İşlenmiş: {result.text}")
print(f"Tokenlar: {result.tokens}")

## 2. Türkçe Karakter Normalizasyonu

In [ ]:
normalizer = TurkishNormalizer()

# Türkçe karakterler
texts = [
    "İstanbul'da güzel bir gün",
    "ÇAĞRI ŞEKER ÖĞRETMEN",
    "ığüşöç karakterleri"
]

for text in texts:
    normalized = normalizer.normalize(text)
    print(f"Orijinal:    {text}")
    print(f"Normalize:   {normalized}")
    print(f"Lowercase:   {normalized.lower()}")
    print()

## 3. Tokenization

In [ ]:
tokenizer = Tokenizer()

text = "Merhaba dünya! Bu bir test metnidir. Python programlama dili."
tokens = tokenizer.tokenize(text)

print(f"Metin: {text}")
print(f"\nTokenlar ({len(tokens)} adet):")
for i, token in enumerate(tokens, 1):
    print(f"  {i}. '{token}'")

## 4. Stop Words Kaldırma

In [ ]:
stopwords = StopWords()

text = "Bu bir test metnidir ve çok önemli bilgiler içerir"
tokens = text.lower().split()

print(f"Orijinal tokenlar: {tokens}")
print(f"\nStop words: {[t for t in tokens if stopwords.is_stopword(t)]}")

filtered = stopwords.remove(tokens)
print(f"\nFiltered tokenlar: {filtered}")

## 5. Lemmatization

In [ ]:
lemmatizer = TurkishLemmatizer()

words = [
    "kitaplar",
    "evlerde",
    "çalışıyorum",
    "gidiyoruz",
    "güzeldir"
]

print("Lemmatization örnekleri:\n")
for word in words:
    lemma = lemmatizer.lemmatize(word)
    print(f"{word:15} -> {lemma}")

## 6. Tam Pipeline

In [ ]:
# Tüm özellikleri aktif et
full_preprocessor = TurkishPreprocessor({
    'lowercase': True,
    'remove_punctuation': True,
    'remove_stopwords': True,
    'lemmatize': True
})

text = """İstanbul'da yaşayan insanlar çok çalışkan ve başarılıdır. 
Onlar her gün işlerine gidiyorlar ve ailelerini mutlu ediyorlar."""

result = await full_preprocessor.process(text)

print(f"Orijinal metin:")
print(text)
print(f"\nİşlenmiş metin:")
print(result.text)
print(f"\nTokenlar ({len(result.tokens)} adet):")
print(result.tokens)
print(f"\nMetadata:")
for key, value in result.metadata.items():
    print(f"  {key}: {value}")

## 7. Batch İşleme

In [ ]:
import time

# Batch preprocessor
batch_preprocessor = TurkishPreprocessor({
    'lowercase': True,
    'remove_punctuation': True,
    'remove_stopwords': True,
    'lemmatize': True,
    'batch_size': 32
})

# Test metinleri
texts = [
    "Python programlama dili çok güçlüdür",
    "Makine öğrenmesi yapay zeka alt dalıdır",
    "Doğal dil işleme metinleri analiz eder",
    "Veri bilimi veriden içgörü çıkarır",
    "Derin öğrenme sinir ağları kullanır"
] * 10  # 50 metin

# Batch işleme
start = time.time()
results = await batch_preprocessor.process(texts)
batch_time = time.time() - start

print(f"Batch işleme:")
print(f"  Metin sayısı: {len(texts)}")
print(f"  Süre: {batch_time:.3f}s")
print(f"  Metin/saniye: {len(texts)/batch_time:.1f}")

# İlk 3 sonucu göster
print(f"\nİlk 3 sonuç:")
for i, result in enumerate(results[:3], 1):
    print(f"\n{i}. {result.text}")
    print(f"   Tokenlar: {result.tokens}")

## 8. Karşılaştırma: Farklı Konfigürasyonlar

In [ ]:
text = "İstanbul'da yaşayan insanlar çok çalışkan ve başarılıdır."

configs = [
    {'name': 'Sadece lowercase', 'config': {'lowercase': True}},
    {'name': 'Lowercase + punctuation', 'config': {'lowercase': True, 'remove_punctuation': True}},
    {'name': 'Lowercase + stopwords', 'config': {'lowercase': True, 'remove_stopwords': True}},
    {'name': 'Tam pipeline', 'config': {
        'lowercase': True,
        'remove_punctuation': True,
        'remove_stopwords': True,
        'lemmatize': True
    }}
]

print(f"Orijinal: {text}\n")

for item in configs:
    preprocessor = TurkishPreprocessor(item['config'])
    result = await preprocessor.process(text)
    
    print(f"{item['name']}:")
    print(f"  {result.text}")
    print(f"  Token sayısı: {len(result.tokens)}")
    print()

## 9. Gerçek Dünya Örneği: Haber Metni

In [ ]:
news_text = """Türkiye'nin başkenti Ankara'da bugün önemli bir toplantı gerçekleştirildi. 
Toplantıya katılan yetkililer, ekonomik gelişmeler hakkında detaylı bilgiler paylaştılar. 
Yapılan açıklamalara göre, önümüzdeki dönemde birçok yeni proje hayata geçirilecek. 
Bu projeler, ülke ekonomisine önemli katkılar sağlayacak ve istihdam artışına yol açacak."""

preprocessor = TurkishPreprocessor({
    'lowercase': True,
    'remove_punctuation': True,
    'remove_stopwords': True,
    'lemmatize': True
})

result = await preprocessor.process(news_text)

print("Haber Metni Analizi\n")
print(f"Orijinal uzunluk: {len(news_text)} karakter")
print(f"İşlenmiş uzunluk: {len(result.text)} karakter")
print(f"Token sayısı: {len(result.tokens)}")
print(f"\nAnahtar kelimeler (ilk 10):")
print(result.tokens[:10])
print(f"\nİşlenmiş metin:")
print(result.text)

## 10. Performans Analizi

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Farklı batch size'lar ile test
batch_sizes = [1, 8, 16, 32, 64]
times = []

test_texts = ["Bu bir test metnidir"] * 100

for batch_size in batch_sizes:
    preprocessor = TurkishPreprocessor({
        'lowercase': True,
        'remove_punctuation': True,
        'batch_size': batch_size
    })
    
    start = time.time()
    await preprocessor.process(test_texts)
    elapsed = time.time() - start
    times.append(elapsed)
    
    print(f"Batch size {batch_size:2d}: {elapsed:.3f}s")

# Grafik
plt.figure(figsize=(10, 6))
plt.plot(batch_sizes, times, marker='o')
plt.xlabel('Batch Size')
plt.ylabel('Time (seconds)')
plt.title('Preprocessing Performance vs Batch Size')
plt.grid(True)
plt.show()

## Sonraki Adımlar

- [Embeddings Notebook](embeddings.ipynb) - Embedding oluşturma
- [Search Notebook](search.ipynb) - Semantik arama
- [API Dokümantasyonu](https://yourusername.github.io/bns-nlp-engine/api/preprocess/)